# 56

In [ ]:
!pip install transformers

# 57

In [ ]:
dataset = [["What music do you like?", "I like Rock music.", 1],
           ["What is your favorite food?", "I like sushi the best", 1],
           ["What is your favorite color?", "I'm going to be a doctor", 0],
           ["What is your favorite song?", "Tokyo olympic game in 2020 was postponed", 0],
           ["Do you like watching TV shows?", "Yeah, I often watch it in my spare time", 1]]

# 58

### 두 개의 BERT 모델 사용 이유
- 각각의 문장을 독립적으로 처리하여 각 문장의 표현을 개별적으로 학습하기 위함.
- 하나의 BERT 모델에서 두 문장을 동시에 처리하는 것보다 각 문장을 개별적으로 학습함으로 문장 간의 관계를 잘 학습할 수 있다.



In [ ]:
from transformers import BertPreTrainedModel, BertConfig, BertModel, BertTokenizer, AdamW
from torch import nn

# 클래스 정의
class BertEnsembleForNextSentencePrediction(BertPreTrainedModel):
  def __init__(self, config, *args, **kwargs):
      super().__init__(config)

      # QA BERT 모델
      self.bert_model_1 = BertModel(config)
      # AQ BERT 모델
      self.bert_model_2 = BertModel(config)

      """
      두 개의 BERT 모델에서 각각 [CLS] 토큰의 은닉 상태를 추출하여 결합.
      각각의 [CLS] 토큰의 은닉 상태는 768차원 벡터이므로, 이를 결합한다. (1536차원, 768 * 2)
      1536차원을 입력받아 2차원 벡터로 변환. (두 문장이 연속적인지 아닌지를 예측하기 위한 로짓(logit)을 생성)

      """
      self.cls = nn.Linear(2 * self.config.hidden_size, 2)
      # 초기 가중치
      self.init_weights()

  def forward(
          self,
          input_ids=None,
          attention_mask=None,
          token_type_ids=None,
          position_ids=None,
          head_mask=None,
          inputs_embeds=None,
          next_sentence_label=None,
  ):
    # 빈 컨테이너 변수 outputs 생성
    outputs = []

    # input_ids 첫번째 입력(문장) 저장
    input_ids_1 = input_ids[0]

    # input_ids 첫번째 입력(문장)의 attention_mask 저장
    attention_mask_1 = attention_mask[0]

    # bert_model_1에 input_ids_1 투입한 결과를 outputs에 순차적으로 저장
    outputs.append(self.bert_model_1(input_ids_1,
                                     attention_mask=attention_mask_1))

    # input_ids 두번째 입력(문장) 저장
    input_ids_2 = input_ids[1]

    # input_ids 두번째 입력(문장)의 attention_mask 저장
    attention_mask_2 = attention_mask[1]

    # bert_model_2에 input_ids_2 투입한 결과를 outputs에 순차적으로 저장
    outputs.append(self.bert_model_2(input_ids_2,
                                     attention_mask=attention_mask_2))

    # outputs에 쌓인 output의 두번째 요소(output[1])을 하나하나 끄집어내어
    # torch.cat()로 토치 텐서 형태로 병합
    # 이를 통해 마지막 은닉층 임베딩 상태를 구함
    """
    outputs는 두 BERT 모델의 출력(튜플 형태)을 담고 있다.
    각 출력은 BERT 모델의 전체 토큰 임베딩과 [CLS] 토큰의 은닉 상태를 포함.
    output[1]은 각 BERT 모델 출력의 [CLS] 토큰의 은닉 상태(벡터)를 나타냄.
    [CLS] 토큰의 은닉 상태는 각 BERT 모델에서 768차원의 벡터로 출력되며, 두 개를 결합하면 1536차원이 됨.
    """
    last_hidden_states = torch.cat([output[1] for output in outputs], dim=1)
    # self.cls 선형함수에 마지막 은닉층 임베딩 상태를 투입하여 로짓 추출
    logits = self.cls(last_hidden_states)

    # 크로스엔트로피 손실(crossentropyloss) 구하기
    # crossentropyloss: https://pytorch.org/docs/stable/nn.html#crossentropyloss
    """
    ignore_index=-1는 손실 함수 계산에서 특정 레이블 값을 무시하도록 설정하는 옵션. (레이블이 -1인 데이터는 손실 계산에서 제외)

    ```
    logits = torch.tensor([[2.0, 0.5], [1.5, 1.0], [1.0, 2.0], [0.0, 0.0]])  # 모델 출력
    labels = torch.tensor([1, 1, 1, -1])  # 레이블, -1은 패딩

    # 손실 함수 정의 (ignore_index=-1 설정)
    loss_fn = nn.CrossEntropyLoss(ignore_index=-1)

    # 손실 계산 (패딩 부분은 무시됨)
    loss = loss_fn(logits, labels)
    print(loss)  # 패딩 부분의 손실은 계산에서 제외

    ```
    logits.view(-1, 2): 로짓 벡터를 (batch_size, 2) 형태로 변환하여 각 예측이 두 개의 클래스(연속 여부)에 대해 로짓 값을 갖도록 함.
    next_sentence_label.view(-1): 레이블을 1차원 형태로 변환하여 각 예측과 일치하게 만듬.
    """
    if next_sentence_label is not None:
      # nn.CrossEntropyLoss( ) 입력 데이터의 마지막 인덱스는 계산에서 제외
      loss_fct = nn.CrossEntropyLoss(ignore_index=-1)
      # logits.view(-1,2)는 열이 두 개 형태로 logits를 정렬
      # next_sentence_label.view(-1)는 행이 하나인 형태(flattening)로 정렬
      next_sentence_loss = loss_fct(logits.view(-1, 2), next_sentence_label.view(-1))
      return next_sentence_loss, logits
    else:
      return logits

#59

[Regularization](https://kh-kim.github.io/nlp_with_deep_learning_blog/docs/reguarlizations)
![](https://kh-kim.github.io/nlp_with_deep_learning_blog/assets/images/1-14/01-overview.png)

![](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*ozLs-feHr73kJTfKL8figA.png)

### L1(lasso) vs L2(ridge)
[L1,L2 정규화 이해](https://medium.com/intuition/understanding-l1-and-l2-regularization-with-analytical-and-probabilistic-views-8386285210fc)

![](https://miro.medium.com/v2/resize:fit:602/0*IgQLm-qC3bBLMHc5.png)
![](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*wTHF3m-JmGaX6tvHcHjxtw.png)


![](https://github.com/acho98/practical-transformers/blob/main/img/img_22.png?raw=true)

<details>

#### L1 정규화
  - 특성 제거에 유용
    - L1 정규화는 가중치를 0으로 만들어 불필요한 특성을 제거하는 데 유리.
    - 특성 선택이나 모델의 해석 가능성을 높이고자 할 때 주로 사용.

#### L2 정규화
  - 모든 특성 값을 작게 만들 때 유용
    - L2 정규화는 모든 가중치를 균일하게 줄이는 방식으로, 가중치가 0이 되는 것을 방지하고 모델의 일반화 성능을 높임.
  - 딥러닝에서 주로 사용
    - 딥러닝 모델에서는 대부분의 특성이 중요할 수 있으므로, L2 정규화를 사용하여 안정적인 학습과 일반화 성능을 높이는 것이 유리
    - L2 정규화는 가중치가 폭발적으로 커지는 것을 방지하여 학습의 안정성을 높임.


#### L1 정규화와 미분 문제
- 미분 문제
  - L1 정규화는 가중치의 절대값을 사용하기 때문에, 0에서의 미분이 정의되지 않는 불연속점이 존재.
  - 예를 들어, ∣w∣의 미분은 𝑤>0 일 때 1, 𝑤<0일 때 −1, 그리고 𝑤=0 일 때는 미분값이 정의되지 않음.

- 서브그래디언트 (Subgradient)
  - 서브그래디언트는 미분이 정의되지 않는 함수에 대해, 미분 대신 사용할 수 있는 대안적인 값.
  - 서브그래디언트는 함수가 볼록일 때 미분이 정의되지 않은 점에서도 근사적으로 기울기를 제공하는 방법.
  - L1 정규화에서, 서브그래디언트는 절대값 함수의 0에서의 기울기를 근사.
  - 예를 들어 ∣𝑤∣의 서브그래디언트는 𝑤>0 일 때 1, w<0 일 때 −1, 그리고 𝑤=0 일 때는 [−1,1]의 범위 내 값을 취할 수 있다.

- 좌표 하강법의 개념
  - 좌표 하강법은 변수별로 최적화하는 과정에서 미분이 가능하면 그라디언트를 사용하고, 미분이 불가능한 지점에서는 서브그래디언트를 사용하는 접근.
  - 서브그래디언트 계산 방식은 좌표 하강법 내에서 사용되는 한 가지 도구
</details>

In [ ]:
import torch
from transformers import AdamW

# 코랩에서 가능한 경우 GPU를 사용하고, 그렇지 않으면 CPU 사용하도록
# 변수 device로 설정을 저장
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 및 config 설정
config = BertConfig()
model = BertEnsembleForNextSentencePrediction(config)

# 모델을 변수 device에서 설정한 대로 GPU 혹은 CPU로 전송
model.to(device)

# 토크니이저 불러오기
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

# 학습률 설정
learning_rate = 1e-5

# 절편과 가중치를 no_decay 변수에 저장
no_decay = ["bias", "LayerNorm.weight"]

# 최적화 함수 그룹 파라미터 설정
optimizer_grouped_parameters = [{
  "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
  }]

# 최적화 함수 설정
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


# 60

In [ ]:
#함수 prepare_data 정의
def prepare_data(dataset, qa=True):
  """
  return 형식:
    input_ids:
    tensor([[ 101, 2009, 2001, 1037, 2204, 2154,  102,    0,    0,    0],
            [ 101, 2001, 2023, 1037, 2965, 2154,  102,    0,    0,    0]])

    attention_masks:
    tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
            [1, 1, 1, 1, 1, 1, 1, 0, 0, 0]])

  qa 조건:
    qa=True:
      데이터셋의 각 항목이 (질문, 답변, 레이블) 순서로 되어 있다고 가정.
    qa=False:
      데이터셋의 각 항목이 (답변, 질문, 레이블) 순서로 되어 있다고 가정.
  """

  # 빈 컨테이너 변수 생성
  input_ids, attention_masks = [], []
  labels = []

  for point in dataset:
    if qa is True:
      # point에 있는 3개의 원소를 앞에 요소부터 q, a, _ 으로 배정
      q, a, _ = point
    else:
      # point에 있는 3개의 원소를 앞에 요소부터 a, q, _ 으로 배정
      a, q, _ = point
    # q와 a를 토크나이저를 통해 인코딩
    encoded_dict = tokenizer.encode_plus(
      q,  # 문장 1 인코딩
      a,  # 문장 2 인코딩
      add_special_tokens=True,  # 특수 토큰인 [CLS]와 [SEP] 생성
      max_length=128,           # max_length 설정Pad & truncate all sentences.
      pad_to_max_length=True,   # max_length까지 패딩 실행
      return_attention_mask=True,  # attention_mask 생성
      return_tensors='pt',      # 파이토치 텐서 타입으로 출력
      truncation=True           # truncation 실행
    )

    # encoded_dict("input_ids")를 컨테이너 변수 input_ids에 순서대로 저장
    input_ids.append(encoded_dict["input_ids"])
    # encoded_dict("attention_mask")를 컨테이너 변수 attention_mask에 순서대로 저장
    attention_masks.append(encoded_dict["attention_mask"])
    # point의 마지막(세번째) 요소(레이블)을 컨테이너 변수 labels에 순서대로 저장
    labels.append(point[-1])
    # 반복문 종료

  # input_ids를 첫번째 축(dim=0), 즉 세로 방향으로 병합
  input_ids = torch.cat(input_ids, dim=0)

  # attention_mask를 첫번째 축(dim=0), 즉 세로 방향으로 병합
  attention_masks = torch.cat(attention_masks, dim=0)

  # 함수의 결과물로 input_ids, attention_mask, lables 출력
  return input_ids, attention_masks, labels

# 61

In [ ]:
import numpy as np
from torch.utils.data import DataLoader, RandomSampler, Dataset, SequentialSampler

# QADataset 클래스 생성
class QADataset(Dataset):

  # input_ids 텐서와 attention_masks 텐서 생성
  def __init__(self, input_ids, attention_masks, labels=None):
    self.input_ids = np.array(input_ids)
    self.attention_masks = np.array(attention_masks)
    # toch.long은 정수(integer) 타입을 의미
    self.labels = torch.tensor(labels, dtype=torch.long)

  def __getitem__(self, index):
    return self.input_ids[index], self.attention_masks[index], self.labels[index]

  def __len__(self):
    return self.input_ids.shape[0]

# 62

In [ ]:
# dataset을 prepare_data 함수에 투입하여 그 결과를 각기
# input_ids_qa, attention_maskes_qa, labels_qa에 저장
input_ids_qa, attention_masks_qa, labels_qa = prepare_data(dataset)

# 위의 세 결과물을 QADataset 클래스에 투입
train_dataset_qa = QADataset(input_ids_qa, attention_masks_qa, labels_qa)

# 맨 윗줄 코드와 동일하나 이번에는 prepare_data 함수에 qa 플래그 값이 False일 때를 적용
input_ids_aq, attention_masks_aq, labels_aq = prepare_data(dataset, qa=False)

# 바로 위 세 결과물을 QADataset 클래스에 투입
train_dataset_aq = QADataset(input_ids_aq, attention_masks_aq, labels_aq)

# train_dataset_qa를 DataLoader로 처리
dataloader_qa =  DataLoader(dataset=train_dataset_qa,
                            batch_size=5,
                            sampler=SequentialSampler(train_dataset_qa))
# train_dataset_aq를 DataLoader로 처리
dataloader_aq =  DataLoader(dataset=train_dataset_aq,
                            batch_size=5,
                            sampler=SequentialSampler(train_dataset_aq))

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


# 63

In [ ]:
# 런타임 20, 30초 이내, 단 GPU 미사용시 런타임 약 8분 소요
# 에포크 횟수 지정
epochs = 30

# 에포크 횟수만큼 반복 루프 실행
for epoch in range(epochs):

  # dataloader_qa와 datalaode_aq 쌍(pair)를 동시에 반복 루프에서 처리
  # enumerate 및 zip으로 두 데이터 쌍을 묶고 반복가능한 순서 부여
  for step, combined_batch in enumerate(zip(dataloader_qa, dataloader_aq)):
    # enumerate로 묶은 두 데이터쌍을 순서대로 batch_1과 batch_2에 저장
    batch_1, batch_2 = combined_batch
    #모델을 학습 모드로 전환
    model.train()

    # 가능한 경우 batch_1과 batch_2의 데이터를 GPU로 전달하고
    # 그렇지 않은 경우 CPU로 전달
    batch_1 = tuple(t.to(device) for t in batch_1)
    batch_2 = tuple(t.to(device) for t in batch_2)

    # 모델에 투입할 변수 inputs의 내용 입력
    inputs = {
        "input_ids": [batch_1[0], batch_2[0]],
        "attention_mask": [batch_1[1], batch_2[1]],
        "next_sentence_label": batch_1[2]
    }
    # 모델에 inputs를 **kwargs 형식(**inputs로 표기)으로 투입
    # 즉, 딕셔너리 타입인 inputs의 키(key)와 키값(value) 모두 입력
    outputs = model(**inputs)

    # 모델의 결과물인 outputs는 튜플 타입으로 출력
    # 그중 첫번째 요소, 즉 outputs[0]을 변수 loss에 저장
    loss = outputs[0]
    # 오차 역전파
    loss.backward()
    # f 문자열을 사용하여 에포크와 손실 출력
    print(f"epoch:{epoch}, loss:{loss}")

    # 가중치 업데이트
    optimizer.step()
    # 다음 번 에포크를 위해 그래디언트(기울기) 초기화
    model.zero_grad()

epoch:0, loss:0.741225004196167
epoch:1, loss:0.7637043595314026
epoch:2, loss:0.6023510694503784
epoch:3, loss:0.5768597722053528
epoch:4, loss:0.579590916633606
epoch:5, loss:0.505245566368103
epoch:6, loss:0.4093991219997406
epoch:7, loss:0.4387204647064209
epoch:8, loss:0.38307610154151917
epoch:9, loss:0.35497426986694336
epoch:10, loss:0.3005308210849762
epoch:11, loss:0.2931199371814728
epoch:12, loss:0.22331778705120087
epoch:13, loss:0.30169934034347534
epoch:14, loss:0.164110004901886
epoch:15, loss:0.21272769570350647
epoch:16, loss:0.11817000061273575
epoch:17, loss:0.14892370998859406
epoch:18, loss:0.09063137322664261
epoch:19, loss:0.09122316539287567
epoch:20, loss:0.061093784868717194
epoch:21, loss:0.05769053101539612
epoch:22, loss:0.035160016268491745
epoch:23, loss:0.041428156197071075
epoch:24, loss:0.023718442767858505
epoch:25, loss:0.016276556998491287
epoch:26, loss:0.01851181499660015
epoch:27, loss:0.014967137947678566
epoch:28, loss:0.013931989669799805
epo

#64

In [ ]:
# dataset을 prepare_data 함수로 처리
input_ids_qa, attention_masks_qa, labels_qa = prepare_data(dataset)
# 위 결과물을 QADataset 클래스로 처리
test_dataset_qa = QADataset(input_ids_qa, attention_masks_qa, labels_qa)

# dataset을 prepare_data 함수로 처리하되 qa 플래그를 False로 적용
input_ids_aq, attention_masks_aq, labels_aq = prepare_data(dataset, qa=False)
# 위 결과물을 QADataset 클래스로 처리
test_dataset_aq = QADataset(input_ids_aq, attention_masks_aq, labels_aq)

# test_dataset_qa를 DataLoader로 처리
dataloader_qa =  DataLoader(dataset=test_dataset_qa,
                            batch_size=16,
                            sampler=SequentialSampler(test_dataset_qa))
# test_dataset_aq를 DataLoader로 처리
dataloader_aq =  DataLoader(dataset=test_dataset_aq,
                            batch_size=16,
                            sampler=SequentialSampler(test_dataset_aq))

# 결과를 담을 빈 컨테이너로서 리스트 변수 생성
complete_outputs, complete_label_ids = [], []

# dataloader_qa와 dataloade_aq를 동시에 반복 루프 작업 실시

for step, combined_batch in enumerate(zip(dataloader_qa, dataloader_aq)):

  # 모델을 eval 모드로 전환
  model.eval()
  # enumerate로 묶은 두 데이터 쌍을 순서대로 batch_1과 batch_2에 저장
  batch_1, batch_2 = combined_batch

  # 가능한 경우 batch_1과 batch2의 데이터를 GPU로 전달하고
  # 그렇지 않은 경우 CPU로 전달
  batch_1 = tuple(t.to(device) for t in batch_1)
  batch_2 = tuple(t.to(device) for t in batch_2)

  # 추론(evaluation) 때는 순전파(forward propagation) 과정만 필요하기에
  # 그래디언트(기울기) 계산이 필요 없음. 아래 코딩 줄은 자동미분을 방지.
  with torch.no_grad():

    # 모델에 투입할 변수 inputs의 내용 입력
    inputs = {
        "input_ids": [batch_1[0], batch_2[0]],
        "attention_mask": [batch_1[1], batch_2[1]],
        "next_sentence_label": batch_1[2]
    }

    # 모델에 inputs를 **kwargs 형식(**inputs로 표기)으로 투입
    # 즉, 딕셔너리 타입인 inputs의 키(key)와 키값(value) 모두 입력
    outputs = model(**inputs)

    # outputs의 첫번째 요소를 tmp_eval_loss에 저장
    # outputs의 두번째 요소를 logits에 저장
    tmp_eval_loss, logits = outputs[:2]

    # 로짓을 CPU로 전달하고 넘파이 값으로 변환
    logits = logits.detach().cpu().numpy()
    # logits에 담긴 로짓값을 축(axis) 1, 즉 가로방향으로 최대값을 갖는 인덱스 추출
    outputs = np.argmax(logits, axis=1)
    # inputs['next_sentence_label']을 CPU로 전달하고 넘파이 값으로 변환
    label_ids = inputs["next_sentence_label"].detach().cpu().numpy()
    # with torch.no_grad(): 구문 종료

  # outputs과 label_ids를 각각의 컨테이너 리스트 변수에 순서대로 저장
  complete_outputs.extend(outputs)
  complete_label_ids.extend(label_ids)

# 최종 결과물 출력
print(complete_outputs, complete_label_ids)

[1, 1, 0, 0, 1] [1, 1, 0, 0, 1]


# 65

In [ ]:
# 이 코드의 설명은 앞의 문제들 주석과 동일
dataset = [["What music do you like?", "I like Rock music.", 1]]

input_ids_qa, attention_masks_qa, labels_qa = prepare_data(dataset)
test_dataset_qa = QADataset(input_ids_qa, attention_masks_qa, labels_qa)

input_ids_aq, attention_masks_aq, labels_aq = prepare_data(dataset, qa=False)
test_dataset_aq = QADataset(input_ids_aq, attention_masks_aq, labels_aq)

dataloader_qa =  DataLoader(dataset=test_dataset_qa,
                            batch_size=16,
                            sampler=SequentialSampler(test_dataset_qa))
dataloader_aq =  DataLoader(dataset=test_dataset_aq,
                            batch_size=16,
                            sampler=SequentialSampler(test_dataset_aq))

complete_outputs, complete_label_ids = [], []

for step, combined_batch in enumerate(zip(dataloader_qa, dataloader_aq)):
  model.eval()
  batch_1, batch_2 = combined_batch

  batch_1 = tuple(t.to(device) for t in batch_1)
  batch_2 = tuple(t.to(device) for t in batch_2)

  with torch.no_grad():
    inputs = {
        "input_ids": [batch_1[0], batch_2[0]],
        "attention_mask": [batch_1[1], batch_2[1]],
        "next_sentence_label": batch_1[2]
    }
    outputs = model(**inputs)
    tmp_eval_loss, logits = outputs[:2]
    logits = logits.detach().cpu().numpy()
    outputs = np.argmax(logits, axis=1)
    label_ids = inputs["next_sentence_label"].detach().cpu().numpy()
  complete_outputs.extend(outputs)
  complete_label_ids.extend(label_ids)

print(complete_outputs, complete_label_ids)

[1] [1]
